# Bureau d'étude : Gestionnaire de réseau électrique (10h)

## a) Objectif de l'étude :
* Connaître les différents éléments d'un réseau électrique
* Comprendre les enjeux d'un calcul de répartition des charges (power-flow)
* Réaliser l'optimisation d'un réseau électrique en service

## b) Modalité d'évaluation :
* Formulaire en ligne à remplir avec les résultats aux questions, un par groupe [lien](#)
* Exercice I (8 points) pour chacune des trois questions un graphique mettant clairement la réponse en évidence sera attendu (à uploader au format png)
* Exercice II (12 points) trouver les set-points qui minimise les frais d'exploitation du réseau, une partie de la note dépendra de votre classement à trouver le plus petit coût.

## c) Organisation :
##### 2 h de cours magistral :
* La structure en tension du réseau,
* La puissance réactive,
* Les equations de power-flow.
##### 1 h prise en main du 'notebook' :
* Illustration du cours sur un réseau simple
* Présentation du problème à étudier
##### 7 h résolution des exercices I et II
* travail en groupe de 2 étudiant.es

## d) Ressources pour aller plus loin :
* [A Gentle Introduction to Power Flow](https://invenia.github.io/blog/2020/12/04/pf-intro/) (en anglais)


-----------------------
# (suite du cours) Illustration sur un réseau simple

![](simple_network.png)

In [16]:
import pandapower as pp  # https://pandapower.readthedocs.io/en/latest/index.html

net = pp.create_empty_network()

# Define nodes (also named bus)
b1 = pp.create_bus(net, vn_kv=110)
b2 = pp.create_bus(net, vn_kv=20)
b3 = pp.create_bus(net, vn_kv=20)

# Add elements of the network
pp.create_ext_grid(net, bus=b1)
pp.create_transformer(net, hv_bus=b1, lv_bus=b2, std_type="25 MVA 110/20 kV")
pp.create_line(net, from_bus=b2, to_bus=b3, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV")
pp.create_load(net, bus=b3, p_mw=13, q_mvar=0)

# Run a power-flow
pp.runpp(net)

# Results
display(net.res_bus)

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-13.175143,-0.873977
1,0.995671,-153.629404,0.000000,0.000000
2,0.985603,-154.162784,13.000000,0.000000


Note comme attendu :
1. la tension descend le long de la ligne comme il y a une consommation de puissance active et réactive
2. le "slack bus" ou "external grid" fourni un peu plus de puissance active et réactive pour couvrir les pertes

In [7]:
display(net.res_line.loc[:, ["loading_percent"]])

,loading_percent
0,90.441645


------------------------------
# Exercice I - Allonger un réseau électrique à une branche
## Objectif :
* Se familiariser avec [PandaPower](https://pandapower.readthedocs.io/en/latest/index.html) pour résoudre un power-flow
* Mettre en pratique les connaissances du cours

## Contexte

![](simple_network.png)

On imagine un réseau moyenne tension composé d'un transformateur 110kV vers 20kV, une ligne aérienne de 2.5km, puis une charge de 13MW.
Contraintes : pour rester dans un fonctionnement normal, la ligne et le transformateur ne doivent pas dépasser un "loading" de 100%.
De même l'amplitude de tension à tous les nœuds doit rester entre 0.95 p.u et 1.05 p.u (i.e., plus ou moins 5%).

## Question :
1. Augmenter la longueur de ligne : quelle longueur de ligne avant l'apparition d'une contrainte en tension ou en loading maximum ?
2. On ajoute une compensation en puissance réactive sur la charge de "q_mvar = -6 MVar" : que devient cette longueur maximum ?
3. Quelle compensation en puissance réactive permet d'atteindre la plus grande longueur de ligne ?

Note : Chaque question devra être répondue par un graphique où la réponse à la question est évidente.

In [10]:
import pandapower as pp

# Create a simple network: external grid 110kV --(b1)-- transformer --(b2)-- line 20kV --(b3)-- load 13MW
def create_and_run_simple_net(line_length_km=2.5, q_mvar=0):
    """
    Note : à ne pas modifier pour garder le même réseau.
    """
    net = pp.create_empty_network()

    # Define nodes (also named bus)
    b1 = pp.create_bus(net, vn_kv=110)
    b2 = pp.create_bus(net, vn_kv=20)
    b3 = pp.create_bus(net, vn_kv=20)

    # Add elements of the network
    pp.create_ext_grid(net, bus=b1)
    pp.create_transformer(net, hv_bus=b1, lv_bus=b2, std_type="25 MVA 110/20 kV")
    pp.create_line(net, from_bus=b2, to_bus=b3, length_km=line_length_km, std_type="NA2XS2Y 1x240 RM/25 12/20 kV")
    pp.create_load(net, bus=b3, p_mw=13, q_mvar=q_mvar)

    # Run a power-flow
    pp.runpp(net)
    return net

In [11]:
net = create_and_run_simple_net(line_length_km=2.5, q_mvar=0)

print("Bus results:")
print("------------")
display(net.res_bus)

print("Line loading results:")
print("---------------------")
display(net.res_line.loc[:, ["loading_percent"]])

Bus results:
------------


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-13.175143,-0.873977
1,0.995671,-153.629404,0.000000,0.000000
2,0.985603,-154.162784,13.000000,0.000000


Line loading results:
---------------------


,loading_percent
0,90.441645


------------------------
# Exercice II - Minimiser les pertes dans un réseau électrique maillé
## Objectif :
* Mixer connaissances physique et optimisation multivariables dans un réseau maillé

## Contexte

![](9bus.png)

On imagine le réseau IEEE 9-Bus comme un territoire à part entière qui cherche à minimiser ses coûts de fonctionnement. Pour cela, le gestionnaire de réseau se doit de sélectionner les générateurs les moins chers tout en respectant les contraintes en tension et 'loading' du réseau.

| Générateur        | prix      |
|-------------------|-----------|
| Ext_grid at bus 1 | 150 €/MWh |
| Gen 2 at bus 2    | 25 €/MWh  |
| Gen 3 at bus 3    | 50 €/MWh  |

L'espace de décision du gestionnaire se restreint à 5 variables : $P_2, v_2, P_3, v_3$ les puissances actives et la tension aux bornes des générateurs 2 et 3 et $v_1$ la tension de "ext_grid" représentant le slack bus. Seule la puissance active est facturée pendant 1 h d'opération, l'équation de coût est alors $coût = P_2 * \pi_2 + P_3 * \pi_3 + |P_{ext grid} * \pi_1|$ (eq. 1) avec $\pi$ le prix du MWh du tableau ci-dessus. À noter qu'une éventuelle puissance excédentaire "exportée" slack bus n'est pas rémunérée.

Par ailleurs la consommation de la charge au bus 7 est incertaine, mais sa variation est régie par une loi de probabilité normale qui suit les paramètres suivants $\mu=0$MW et $\sigma^2=12.5$MW.

Enfin le gestionnaire de réseau tolère des conditions au dela de **100% de loading** sur les lignes et **une tension variant de 0.9 pu à 1.1pu** pendant 5% du temps, ce qui lui permet de réduire ses coûts, en évitant de prévoir pour le pire scénario. Pour pénaliser un fonctionnement hors des contraintes au dela de 5% du temps, chaque pourcentage supplémentaire est facturé 1000 € (par exemple, si 7% du temps les contraintes sont dépassées soit 2% de trop, alors une pénalité de $2 \times 1000 = 2000€$ s'ajoute au coût de l'eq. 1).

> **_Notes :_** les contraintes en tensions ont été relaxées à $\pm 10$% comparé à l'exercice I où elles étaient à $\pm 5$%.

## Question
1. Minimiser le coût de l'énergie pour la gestion du réseau électrique en donnant les variables $P_2, v_2$ et $P_3, v_3$ pour les générateurs aux bus 2 et 3 et $v_1$ pour la tension au slack bus 1.

> **_Réponse à la question :_** "26 k€, mais il est à priori possible de faire mieux."

&nbsp;

---
**_Astuces pour décomposer le problème :_**
* Sans tenir compte des contraintes en tension et 'loading', ni de la variation de load 7, quel est le coût minimum ?
* Quel coût en tenant compte des contraintes réseaux, mais sans la variation de load 7 ?
* Prenez ensuite en compte les incertitudes sur la load 6...
---


In [1]:
import pandapower as pp

def create_and_run_9bus(slack1_v=1.0, gen2_mw=0, gen2_v=1.0, gen3_mw=0, gen3_v=1.0, load7_variation=0):
    """
    Note : à ne pas modifier pour garder le même réseau.
    """
    # Create the network
    # -----------------------------------------
    net = pp.networks.case9()

    # Increase capacity of transformer
    net.line.loc[0, "max_i_ka"] = 1.0
    net.line.loc[3, "max_i_ka"] = 1.0
    net.line.loc[6, "max_i_ka"] = 1.0

    # Lines around slack bus
    net.line.loc[1, "max_i_ka"] = 1.0
    net.line.loc[8, "max_i_ka"] = 1.0

    # Lines around 2nd gen
    net.line.loc[2, "max_i_ka"] = 0.5
    net.line.loc[4, "max_i_ka"] = 0.5

    # Line around 1st gen
    net.line.loc[5, "max_i_ka"] = 0.25
    net.line.loc[7, "max_i_ka"] = 0.25

    # Load demand
    net.load.loc[0, "p_mw"] = 150
    net.load.loc[0, "q_mvar"] = -150

    net.load.loc[1, "p_mw"] = 300
    net.load.loc[1, "q_mvar"] = 75

    net.load.loc[2, "p_mw"] = 150
    net.load.loc[2, "q_mvar"] = 50
    # -----------------------------------------

    # Slack bus voltage
    net.ext_grid.loc[0, "vm_pu"] = slack1_v

    # Change generator set-points
    # Generator 1
    net.gen.loc[0, "p_mw"] = gen2_mw
    net.gen.loc[0, "vm_pu"] = gen2_v

    # Generator 2
    net.gen.loc[1, "p_mw"] = gen3_mw
    net.gen.loc[1, "vm_pu"] = gen3_v

    # Load variation
    net.load.loc[1, "p_mw"] += load7_variation

    pp.runpp(net)
    return net

In [5]:
net = create_and_run_9bus(slack1_v=1.0, gen2_mw=125, gen2_v=1.0, gen3_mw=200, gen3_v=1.0, load7_variation=0)

print("Bus results:")
print("------------")
display(net.res_bus)

print("Line loading results:")
print("---------------------")
display(net.res_line.loc[:, ["loading_percent"]])

Bus results:
------------


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-289.069652,17.254515
1,1.000000,-12.937340,-125.000000,-37.229320
2,1.000000,-6.222188,-200.000000,0.278609
3,1.023572,-9.361888,0.000000,0.000000
4,1.095169,-16.320725,150.000000,-150.000000
5,1.007007,-12.905677,0.000000,0.000000
6,0.939376,-23.162607,300.000000,75.000000
7,0.979851,-17.510472,0.000000,0.000000
8,0.977633,-16.851815,150.000000,50.000000


Line loading results:
---------------------


,loading_percent
0,48.461301
1,27.132510
2,26.366515
3,33.469613
4,62.137075
5,96.214344
6,21.826570
7,12.541943
8,27.343052
